In [1]:
import pandas as pd
import numpy as np

In [2]:
#loading csv files to clean data
circuits = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\circuits.csv")
constructor_results = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\constructor_results.csv")
constructor_standings = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\constructor_standings.csv")
constructors = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\constructors.csv")
driver_standings = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\driver_standings.csv")
drivers = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\drivers.csv")
lap_times = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\lap_times.csv")
pit_stops = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\pit_stops.csv")
qualifying_info = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\qualifying.csv")
races = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\races.csv")
results = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\results.csv")
seasons = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\seasons.csv")
sprint_results = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\sprint_results.csv")
status = pd.read_csv(r"C:\Users\shrig\F1 prediction model\csv files\status.csv")

In [102]:
#checking table sizes of each file
files = [circuits, constructor_results, constructor_standings, constructors, driver_standings, drivers, lap_times, pit_stops, qualifying_info, races, results, seasons,
        sprint_results, status]

for file in files:
    print(len(file), "rows")

77 rows
12625 rows
13391 rows
212 rows
34863 rows
861 rows
589081 rows
11371 rows
10494 rows
1125 rows
26759 rows
75 rows
360 rows
139 rows


In [105]:
#merging necessary tables using .merge()
#merging results with drivers 
merged_df = results.merge(drivers, on = 'driverId', how = 'left')

#add constructors
merged_df = merged_df.merge(constructors, on = 'constructorId', how = 'left', suffixes = ('', '_constructor'))

#add races, dealing with duplicate columns('url')
merged_df = merged_df.merge(races, on = 'raceId', how = 'left', suffixes=('', '_race'))

#add circuits
merged_df = merged_df.merge(circuits, on = 'circuitId', how = 'left', suffixes = ('', '_circuits'))

#add qualifying
merged_df = merged_df.merge(qualifying_info, on=['raceId', 'driverId'], how='left', suffixes = ('', '_qualifying'))

#add status
merged_df = merged_df.merge(status, on = 'statusId', how='left', suffixes = ('', '_status'))

merged_df.head()

,resultId,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,...,alt,url_circuits,qualifyId,constructorId_qualifying,number,position_qualifying,q1,q2,q3,status
0,1,18,1,1,22,1,1,1,1,10.0,...,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,1.0,1.0,22.0,1.0,1:26.572,1:25.187,1:26.714,Finished
1,2,18,2,2,3,5,2,2,2,8.0,...,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,5.0,2.0,3.0,5.0,1:25.960,1:25.518,1:27.236,Finished
2,3,18,3,3,7,7,3,3,3,6.0,...,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,7.0,3.0,7.0,7.0,1:26.295,1:26.059,1:28.687,Finished
3,4,18,4,4,5,11,4,4,4,5.0,...,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,12.0,4.0,5.0,12.0,1:26.907,1:26.188,\N,Finished
4,5,18,5,1,23,3,5,5,5,4.0,...,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,3.0,1.0,23.0,3.0,1:25.664,1:25.452,1:27.079,Finished


In [107]:
#cleaning and organizing dataset
#checking merge quality
print(merged_df.shape)
print(merged_df.isnull().sum().sort_values(ascending=False).head(15))
merged_df.duplicated().sum()

(26759, 63)
q3                          16311
q2                          16287
q1                          16265
position_qualifying         16265
number                      16265
constructorId_qualifying    16265
qualifyId                   16265
resultId                        0
fp1_time                        0
fp3_time                        0
fp3_date                        0
fp2_time                        0
fp2_date                        0
date                            0
fp1_date                        0
dtype: int64


0

In [109]:
#dropping older race data with no qualifying to provide accurate historical information
clean_df = merged_df.dropna(subset=['q1', 'q2', 'q3'])
clean_df = clean_df.rename(columns={'name': 'constructor_name'}) #renaming column name to reflect accurate info--> constructor name
print(clean_df.shape)

(10448, 63)


In [111]:
#function to convert lap time into numeric value
def lap_time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    try:
        mins, secs = time_str.split(":")
        return float(mins) * 60 + float(secs)
    except:
        return None

#function to covert lap time into seconds format
def seconds_to_lap_time(seconds):
    if pd.isna(seconds):
        return None
    try:
        mins = int(seconds // 60)
        secs = seconds % 60
        return f"{mins}:{secs:06.3f}"
    except:
        return None

In [113]:
print(clean_df.columns.tolist())

['resultId', 'raceId', 'driverId', 'constructorId', 'number_x', 'grid', 'position', 'positionText', 'positionOrder', 'points', 'laps', 'time', 'milliseconds', 'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId', 'driverRef', 'number_y', 'code', 'forename', 'surname', 'dob', 'nationality', 'url', 'constructorRef', 'constructor_name', 'nationality_constructor', 'url_constructor', 'year', 'round', 'circuitId', 'name_race', 'date', 'time_race', 'url_race', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'circuitRef', 'name_circuits', 'location', 'country', 'lat', 'lng', 'alt', 'url_circuits', 'qualifyId', 'constructorId_qualifying', 'number', 'position_qualifying', 'q1', 'q2', 'q3', 'status']


In [115]:
#creating new column using key fastestLapTime to convert from string object to numeric value for later analysis
clean_df['fastestLapTimeNumeric'] = clean_df['fastestLapTime'].apply(lap_time_to_seconds)

#creating new column using key fastestLapTime to format time properly for later visualization
clean_df['fastestLapTimeFormatted'] = clean_df['fastestLapTimeNumeric'].apply(seconds_to_lap_time)

# List of columns to keep
cols_to_keep = [
    'raceId', 'year', 'round', 'date',                                                    # Race info
    'driverId', 'forename', 'surname',                                                   # Driver info
    'constructorId', 'constructor_name',                                                 # Constructor info
    'grid', 'points', 'laps', 'fastestLapTimeNumeric', 'fastestLapTimeFormatted',        # Race results
    'q1', 'q2', 'q3', 'position_qualifying',                                             # Qualifying
    'status', 'positionOrder',                                                           # Status & finishing position
    'circuitId', 'name_circuits', 'location', 'country'                                  # Circuit info
]

final_df = clean_df[cols_to_keep].copy()
#final data frame
print(final_df.shape)
final_df.head()

(10448, 24)


,raceId,year,round,date,driverId,forename,surname,constructorId,constructor_name,grid,...,q1,q2,q3,position_qualifying,status,positionOrder,circuitId,name_circuits,location,country
0,18,2008,1,2008-03-16,1,Lewis,Hamilton,1,McLaren,1,...,1:26.572,1:25.187,1:26.714,1.0,Finished,1,1,Albert Park Grand Prix Circuit,Melbourne,Australia
1,18,2008,1,2008-03-16,2,Nick,Heidfeld,2,BMW Sauber,5,...,1:25.960,1:25.518,1:27.236,5.0,Finished,2,1,Albert Park Grand Prix Circuit,Melbourne,Australia
2,18,2008,1,2008-03-16,3,Nico,Rosberg,3,Williams,7,...,1:26.295,1:26.059,1:28.687,7.0,Finished,3,1,Albert Park Grand Prix Circuit,Melbourne,Australia
3,18,2008,1,2008-03-16,4,Fernando,Alonso,4,Renault,11,...,1:26.907,1:26.188,\N,12.0,Finished,4,1,Albert Park Grand Prix Circuit,Melbourne,Australia
4,18,2008,1,2008-03-16,5,Heikki,Kovalainen,1,McLaren,3,...,1:25.664,1:25.452,1:27.079,3.0,Finished,5,1,Albert Park Grand Prix Circuit,Melbourne,Australia


In [117]:
#save cleaned data to csv
final_df.to_csv('f1_cleaned_data.csv', index=False)